In [38]:
#!pip install permanent

In [39]:
import numpy as np
from numpy import sqrt, exp, cos, sin, cosh, sinh, conj
from numpy.linalg import qr
import math
from collections import Counter
from permanent import permanent
from functools import cache

In [40]:
from typing import Tuple
# TODO: See pbs. Should we add type annotations?

# Simulation Model: Threshold Detectors and Squeezed States

### Measurement

In [41]:
DEFAULT_MEAS_THRESH = 1.9494

#### h_or_v_detector

In [42]:
def h_or_v_detector(a: np.ndarray, gamma=DEFAULT_MEAS_THRESH):
    return np.bitwise_or(abs(a[0,:]) > gamma, abs(a[1,:]) > gamma)

#### norm_detector

In [43]:
def norm_detector(a: np.ndarray, gamma=DEFAULT_MEAS_THRESH):
    return np.linalg.norm(a, axis=1) > gamma

In [44]:
def many_mode_detector(a, gamma=DEFAULT_MEAS_THRESH):
    return abs(a) > gamma

#### meas_all_2

In [45]:
def meas_all_2(*vargs, gamma=DEFAULT_MEAS_THRESH, detect_model=h_or_v_detector):
    return np.array(list(map(lambda a: detect_model(a, gamma), vargs)))

In [46]:
def meas_all():
    pass

#### get_coincidence_count

In [47]:
def get_coincidence_count(*vargs):
    '''Returns the number of times all provided detectors simultanously clicked.'''
    detections = np.array(vargs)
    return np.count_nonzero(np.sum(detections, axis=0) == len(vargs))

#### get_all_coincidence_counts

In [48]:
def get_all_coincidence_counts(*detections):
    '''Counts the number of times every possible detection event occurs
    A detection event is a bitstring with 1's at the indices of all detectors that 
    clicked and 0's at the indices of all detectors which did not.
    Returns a Counter object with detection events (as tuples) mapped to counts.'''
    if len(detections) > 1:
        detections = np.array(detections)
    return Counter(map(tuple, detections.T))

#### print_all_coincidence_counts

In [49]:
def print_all_coincidence_counts(*detections):
    if len(detections) > 1:
        detections = np.array(detections)
    coincidence_counts = get_all_coincidence_counts(detections)
    m = detections.shape[0]
    for i in range(2**m):
        bitvector = np.frombuffer(np.binary_repr(i, width=m).encode(), dtype='S1').astype(int)
        detectors = np.binary_repr(i, width=m) # ', '.join(np.flatnonzero(bitvector).astype(str))
        print(detectors, coincidence_counts[tuple(bitvector)])


### State Preparation

#### zpf

In [50]:
def zpf(N: int, M=1, sigma=1/sqrt(2)):
    '''Zero-point field
    N is the number of samples
    M is the number of modes
    sigma is the standard deviation of the quantum noise
    sigma = 1/sqrt(2) corresponds to the vacuum state
    sigma > 1/sqrt(2) corresponds to a thermal state
    sigma = 0 corresponds to a classical (i.e., no ZPF) state'''
    
    return sigma * (np.random.normal(size=(M,N)) + 1j * np.random.normal(size=(M,N))) / sqrt(2)

#### laser

In [51]:
def laser(N: int, alphaH=1, alphaV=0):
    '''Laser
    alphaH and alphaV are each complex numbers'''

    # scale of vacuum fluctations
    sigma0 = 1/sqrt(2)

    # input random variables for the entanglement source
    zH = zpf(N, 1, sigma0)
    zV = zpf(N, 1, sigma0)

    aH = alphaH + zH
    aV = alphaV + zV

    return np.concatenate((aH, aV))

#### ent

In [52]:
def ent(N: int, r: float, t=1, phase=0):
    '''Entanglement source
    N is the number of samples
    r is the squeezing strength (non-negative)
    t (type) is 1 or 2
    phase is in degrees.'''

    # convert degrees to radians
    phase = phase * np.pi/180
    
    #input random variables for the entanglement source
    z1H = zpf(N, 1)
    z1V = zpf(N, 1)
    z2H = zpf(N, 1)
    z2V = zpf(N, 1)

    if t == 1:
        aH = cosh(r)*z1H + sinh(r)*conj(z2H)
        aV = cosh(r)*z1V + exp(1j*phase)*sinh(r)*conj(z2V)
        bH = cosh(r)*z2H + sinh(r)*conj(z1H)
        bV = cosh(r)*z2V + exp(1j*phase)*sinh(r)*conj(z1V)
    elif t == 2:
        aH = cosh(r)*z1H + sinh(r)*conj(z2V)
        aV = cosh(r)*z1V + exp(1j*phase)*sinh(r)*conj(z2H)
        bH = cosh(r)*z2H + exp(1j*phase)*sinh(r)*conj(z1V)
        bV = cosh(r)*z2V + sinh(r)*conj(z1H)
    else:
        print(f"{t} is not a valid type.")
        return

    return np.concatenate((aH, aV)), np.concatenate((bH, bV))

### Gates 

Filters, Waveplates, and Beamsplitters

#### ndf (neutral density filter)

In [53]:
def ndf(a: np.ndarray, d=10):
    '''Neutral density filter
    d is the optical density (a non-negative number)'''

    if d < 0:
        raise ValueError("Optical density d must be non-negative.")

    return 10**(-d/2) @ a + (1-10**(-d/2)) @ zpf(a.shape[1], 2)

#### hwp

In [54]:
def hwp(a: np.ndarray, theta=0):
    '''Half-wave plate
    a is a 2 x N complex matrix
    theta is the fast-axis angle in degrees'''

    # convert degrees to radians
    theta = theta * np.pi/180

    u = np.array([
        [cos(2*theta), sin(2*theta)], 
        [sin(2*theta), -cos(2*theta)]])
        
    return u @ a

#### qwp

In [55]:
def qwp(a: np.ndarray, theta=0):
    '''Quarter-wave plate
    a is a 2 x N complex matrix
    theta is the fast-axis angle in degrees'''

    # convert degrees to radians
    theta = theta * np.pi/180

    u = np.array([
        [cos(theta)^2 + 1j*sin(theta)^2, (1-1j)*cos(theta)*sin(theta)], 
        [(1-1j)*cos(theta)*sin(theta), sin(theta)^2 + 1j*cos(theta)^2]])
        
    return u @ a

#### polarizer

In [56]:
def polarizer(a: np.ndarray, theta=0, phi=0):
    '''Polarizer
    a is a 2 x N complex matrix
    theta, phi are in degrees'''

    # convert degrees to radians
    theta = theta * np.pi/180
    phi = phi * np.pi/180

    # make projector p
    bra = np.array([cos(theta), exp(1j*phi)*sin(theta)])
    p = bra.T @ bra

    return p @ a + (np.eye(2) - p) @ zpf(a.shape[1], 2)

#### bs

In [57]:
def bs(a: np.ndarray, b: np.ndarray, r=1/sqrt(2)):
    '''Beam splitter
    a and b are each 2 x N complex matrices
    r is the reflectance (0 <= r <= 1)'''

    #TODO: where are the defaults / vacuum state logic? (see polarizing beam splitter)

    t = sqrt(1-r^2)

    out = np.kron(np.array([[t, r], [r, -t]]), np.eye(2)) @ np.concatenate((a, b))

    return out[0:2,:], out[2:4,:]

    # TODO: why not just 
    # u = np.array([[t, r], [r, -t]])
    # return u @ a, u @ b

#### pbs

In [58]:
HV = np.array([[1, 0, 0, 0], 
               [0, 0, 0, 1], 
               [0, 0, 1, 0], 
               [0, 1, 0, 0]])

DA = np.array([[1, 1, 1, -1],
               [1, 1, -1, 1], 
               [1, -1, 1, 1], 
               [-1, 1, 1, 1]])

RL = np.array([[1, -1j, 1, 1j], 
               [1j, 1, -1j, 1],
               [1, 1j, 1, -1j],
               [-1j, 1, 1j, 1]])

In [59]:
def pbs(
    a: np.ndarray = None, 
    b: np.ndarray = None, 
    basis=HV
    ) -> Tuple[np.ndarray, np.ndarray]:
    """Polarizing beam splitter
    a and b are each 2 x N complex matrices
    basis is a 4 x 4 complex matrix"""

    if a is None and b is None:
        raise ValueError("At least one input beam must be specified.")
    elif a is None:
        a = zpf(b.shape[1], 2)
    elif b is None:
        b = zpf(a.shape[1], 2)
        
    out = basis @ np.concatenate((a, b))
    
    return out[0:2,:], out[2:4,:]

### Tests

In [60]:
def check_expectations(arr: np.ndarray):
    # avg z
    exp_val = arr.mean()

    # avg z^T z for every column z in arr
    # if statement optimizes speed
    trans = np.trace((arr @ arr.T if arr.shape[0] < arr.shape[1] else arr.T @ arr)) / arr.shape[1]

    # avg z^H z for every column z in arr
    herm = np.linalg.norm(arr, axis=0).mean()
    
    print(f"E[z] = {exp_val} should be 0.")
    print(f"E[z^T z] = {trans} should be 0.")
    print(f"E[z^H z] = {herm} should be 1.")

In [61]:
vac = zpf(1000000, 2)
check_expectations(vac)

E[z] = (0.0001915931435885348-0.00021093875322136926j) should be 0.
E[z^T z] = (0.0008254923848978945-0.0002984104040788904j) should be 0.
E[z^H z] = 0.9407454774998835 should be 1.


In [62]:
a = zpf(100,2)
print(a)
print(h_or_v_detector(a, 1))

[[-0.56890354+0.06281114j  0.25923834+0.51882647j  0.48185152-0.28044394j
  -0.36857187+0.58534198j  0.2253318 +1.11971602j  0.16707334+0.52213306j
   0.26747921+0.38968854j  0.73640951+0.65070456j  0.06986965-0.85426454j
   1.44184997-0.40184897j  0.36377321-0.71723694j  0.02313719-0.39319058j
  -0.06714556-0.29100795j  0.31840065+0.09258718j -0.14533781-0.31820811j
  -0.31253402+0.11315622j  0.31829844-0.74823982j -0.16491556-0.36130202j
  -0.44945305+0.06289855j -0.25117474+0.15215717j  0.22694226-0.04529597j
  -0.020472  -1.36173256j -0.10963532-0.58957706j -0.55580509-0.23760818j
   0.1270176 -0.773615j   -1.11272659+0.35549847j  0.58320232+0.00480145j
  -0.3021795 -0.23446572j  0.02180533+0.02959898j -0.07195311-0.55865616j
   0.13867278+0.04199859j -0.11571669+0.61510173j  0.85016422+0.64236085j
  -0.4238627 -0.81859526j -0.30672905+0.88091992j  0.30074714+0.32743796j
  -1.32036848-0.86484052j  0.13277471+0.13809445j -0.34997341+0.37124012j
   0.57297099+0.53467332j -0.02205803+

In [63]:
l,r = ent(100000, 2)
lzpf = zpf(100000, 2)
rzpf = zpf(100000, 2)

In [64]:
lt, lr = pbs(l, lzpf)
rt, rr = pbs(r, rzpf)

dlt, dlr, drt, drr = meas_all_2(lt, lr, rt, rr)
assert(np.array_equal(h_or_v_detector(lt), dlt))

coincidence_counts = get_all_coincidence_counts(dlt, dlr, drt, drr)
dlt_and_drt = sum([coincidence_counts[event] for event in [(1,0,1,0), (1,0,1,1), (1,1,1,0), (1,1,1,1)]])
assert(get_coincidence_count(dlt,drt) == dlt_and_drt)

print_all_coincidence_counts(dlt, dlr, drt, drr)


AttributeError: 'tuple' object has no attribute 'T'

### Scrap Work

Below this point is temporary test code meant to be deleted.

In [ ]:
from scipy.stats import norm

distr = norm()
reals = distr.rvs(size=(50,100))
imags = distr.rvs(size=(50,100))
print(reals + 1j * imags)

[[-1.71123873+1.03203358j  0.25290903-0.20229517j -1.03858304-1.94695635j
  ... -0.65685099+0.50877123j  1.15537025+0.52594587j
  -0.80333666+0.24437439j]
 [-0.17166034-0.92917247j  0.37375949+0.09067783j -0.85774042+0.38727677j
  ... -0.74151754+1.58133976j  0.00605111-0.10305549j
   0.15554875+0.2018517j ]
 [-0.62026678+0.65516664j  0.05945093-2.02463451j -1.40302085-1.64506059j
  ... -2.59287403+1.0777941j  -0.80680869-0.25058956j
  -0.68023773-1.12981556j]
 ...
 [-1.09497697+1.33154372j  0.20440019-0.6111026j  -0.02520246-0.53924788j
  ... -0.15793673-0.95605601j -0.78757326+1.14971098j
  -1.77271476+1.12320909j]
 [ 0.2072372 -0.17931669j -0.90903383+0.58306946j  1.82195125+1.62878906j
  ...  0.16318993-0.37836838j  0.30660305+2.13509794j
   0.40238808-0.50211981j]
 [ 0.34467285+0.63552983j  0.07088629-0.12504365j  0.41811054+1.01116145j
  ... -0.17981574-0.60115403j -1.50529948+0.96127839j
  -1.61983969+0.89332218j]]


In [ ]:
import re
def f(s):
    s = re.sub('\s+',' ', s)
    a = [i.strip().split(' ') for i in s.split(';')]
    mat = ', '.join([f'[{", ".join(b)}]' for b in a])
    print(f'np.array([{mat}])')

In [ ]:
f('''cos(theta)^2 + 1j*sin(theta)^2 (1-1j)*cos(theta)*sin(theta) (1-1j)*cos(theta)*sin(theta)   sin(theta)^2 + 1j*cos(theta)^2''')

np.array([[cos(theta)^2, +, 1j*sin(theta)^2, (1-1j)*cos(theta)*sin(theta), (1-1j)*cos(theta)*sin(theta), sin(theta)^2, +, 1j*cos(theta)^2]])


In [ ]:
a = np.array([[1j,2],[3,4]])
a.mean()

(2.25+0.25j)

In [ ]:
arr = zpf(1000, 2, 1)
# %timeit np.trace(arr.T @ arr) 
# %timeit np.trace(arr @ arr.T)
%timeit np.trace((arr @ arr.T if arr.shape[0] < arr.shape[1] else arr.T @ arr))
print(np.trace(arr.T @ arr), np.trace(arr @ arr.T))

24.9 µs ± 3.9 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
(-52.81966246791029+2.3665642474168322j) (-52.819662467910504+2.366564247416818j)


In [ ]:
print(np.eye(2) * np.ones((2,2)))           # elementwise product
print(np.eye(2) @ np.ones((2,2)))           # matrix product
print(np.kron(np.eye(2), np.ones((2,2))))   # kronecker product

[[1. 0.]
 [0. 1.]]
[[1. 1.]
 [1. 1.]]
[[1. 1. 0. 0.]
 [1. 1. 0. 0.]
 [0. 0. 1. 1.]
 [0. 0. 1. 1.]]


In [ ]:
def temp():
    return [1,2,3]

a,b,c = temp()
print(a,b,c)

1 2 3


# Simulation Model: QM

#### Fock basis states

In [ ]:
def is_fock_state(np_arr):
    return all([i.is_integer() for i in np_arr])

In [ ]:
@cache
def get_basis_states(n, m):
  '''Returns a numpy array of all (n + m - 1 Choose n) Fock basis 
  states with n particles in m modes.'''

  if not isinstance(n, int) or n < 0:
    raise ValueError("n must be a non-negative integer")
  if not isinstance(m, int) or m < 1:
    raise ValueError("m must be a positive integer")

  if m == 1:
    return np.array([[n]], dtype=int)
  if n == 1:
    return np.eye(m, dtype=int)

  states = None
  for i in range(0,n+1):
    sub_states = get_basis_states(n-i, m-1)
    first_mode = np.full((sub_states.shape[0],1), i)
    new_states = np.concatenate((first_mode, sub_states), axis=1)
    if i == 0:
      states = new_states
    else:
      states = np.concatenate((states, new_states))
  return states
     

#### HilbertSpaceUnitary

Recall 

$$\langle S | \varphi (U) | T \rangle = \frac{Per(U_{S,T})}{\sqrt{s_1! \dots s_m! t_1! \dots t_m!}}$$

See pg 95 of https://www.scottaaronson.com/qisii.pdf for more info.

In [ ]:
class HilbertSpaceUnitary(object):
  '''Converts the m x m unitary of a linear optical network to 
  the larger M x M unitary acting on the entire Hilbert space,
  where M := n+m-1 choose n. Lazily computes entries as needed.'''

  def __init__(self, U, n):
    self.U = U # U should be an m x m unitary
    self.n = n
    self.entries = {}
    self.factorial = np.vectorize(math.factorial) 
    # Assuming few collisions, this should be fine. With high photon counts, we 
    # might consider memozing our factorial function.

  def get_entry(self, S, T):
    '''This gives the entry <S|\phi(U)|T>, that is,
    the amplitude of a T to S transition.
    S and T must both be Fock states.'''

    if np.sum(S) != self.n or np.sum(T) != self.n:
      raise ValueError(f"Fock states must have {self.n} particles.")
    if not is_fock_state(S):
      raise ValueError("S must be a Fock state.")
    if not is_fock_state(T):
      raise ValueError("T must be a Fock state.")

    key = (S.tobytes(),T.tobytes())
    if key not in self.entries:
      U_ST = np.repeat(np.repeat(self.U, S, axis=0), T, axis=1).astype(complex)
      self.entries[key] = ( 
          permanent.permanent(U_ST) 
          / sqrt(np.prod(self.factorial(S)))
          / sqrt(np.prod(self.factorial(T)))
      ) # watch out for multiplication overflows on normalization factor
    return self.entries[key]
  
  def __get_item__(self, S_and_T):
    S,T = S_and_T
    return self.get_entry(S,T)

#### get_hilbert_space_unitary_matrix

In [ ]:
def get_hilbert_space_unitary_matrix(U, n, p=False):
  '''Converts the m x m unitary of a linear optical network to 
  the larger M x M unitary acting on the entire Hilbert space.
  U is an m x m unitary (m is number of modes)
  n is number of particles
  p determines whether or not to print the matrix
  M := n+m-1 choose n'''
  m = U.shape[0]
  basis_states = get_basis_states(n,m)
  hsu = HilbertSpaceUnitary(U, n)
  M = basis_states.shape[0]
  hsu_matrix = np.zeros((M,M), dtype=complex)
  for i in range(M):
    for j in range(M):
      hsu_matrix[i,j] = hsu.get_entry(basis_states[i], basis_states[j])
  if p:
    rounded_hsu = np.around(hsu_matrix,2)
    for i in range(M):
      print(basis_states[i], end=':\t')
      print(rounded_hsu[i])
  return hsu_matrix

#### qr_haar

In [ ]:
# Source: https://pennylane.ai/qml/demos/tutorial_haar_measure.html

def qr_haar(m):
    """Generate a Haar-random matrix using the QR decomposition.
    m is the number of modes."""
    # Step 1
    A, B = np.random.normal(size=(m, m)), np.random.normal(size=(m, m))
    Z = A + 1j * B

    # Step 2
    Q, R = qr(Z)

    # Step 3
    Lambda = np.diag([R[i, i] / np.abs(R[i, i]) for i in range(m)])

    # Step 4
    return np.dot(Q, Lambda)

### Tests

In [ ]:
get_basis_states(3,100)

array([[0, 0, 0, ..., 0, 0, 3],
       [0, 0, 0, ..., 0, 1, 2],
       [0, 0, 0, ..., 0, 2, 1],
       ...,
       [2, 0, 0, ..., 0, 1, 0],
       [2, 0, 0, ..., 0, 0, 1],
       [3, 0, 0, ..., 0, 0, 0]])

In [ ]:
U = qr_haar(100) #np.array([[0, 1], [1, 0]], dtype=complex)

#print(U)

UH = HilbertSpaceUnitary(U, 1)

zers = np.zeros(100, dtype=int)
zert = np.zeros(100, dtype=int)

zers[0] = 1
zert[0] = 1

abs(UH.get_entry(zers, zert))**2

0.0003193031906318097

In [ ]:
hadamard = np.array([[1, 1], [1, -1]]) / sqrt(2)
hsu_had = get_hilbert_space_unitary_matrix(hadamard, 1, True)

m = 6
U = qr_haar(m)
hsu_haar = get_hilbert_space_unitary_matrix(U, 1, True)

[1 0]:	[0.71-0.j 0.71-0.j]
[0 1]:	[ 0.71-0.j -0.71+0.j]
[1 0 0 0 0 0]:	[ 0.24-0.07j -0.32-0.25j -0.05+0.48j -0.23+0.11j -0.17-0.22j -0.63+0.03j]
[0 1 0 0 0 0]:	[ 0.26+0.18j -0.16+0.29j  0.29-0.42j -0.15-0.22j  0.52-0.29j -0.32+0.09j]
[0 0 1 0 0 0]:	[-0.19-0.51j  0.18+0.24j  0.41-0.04j -0.25+0.11j -0.12-0.11j -0.11-0.58j]
[0 0 0 1 0 0]:	[ 0.16-0.11j  0.07-0.46j  0.12-0.16j -0.58+0.37j  0.3 +0.3j   0.19+0.13j]
[0 0 0 0 1 0]:	[-0.19-0.02j  0.32-0.47j  0.42+0.21j  0.2 -0.13j  0.15-0.53j  0.14+0.2j ]
[0 0 0 0 0 1]:	[ 0.65+0.21j  0.31-0.05j  0.27-0.1j   0.37+0.35j -0.23+0.06j -0.06-0.18j]


# Experiments

In [ ]:
def spoof_haar(num_src, N=100):
    m = 2*num_src
    ents = [ent(N, r=1) for i in range(num_src)]
    l_ents = np.concatenate(tuple([ent[0] for ent in ents]))
    r_ents = np.concatenate(tuple([ent[1] for ent in ents]))
    haar_U = qr_haar(m)
    r_ents = haar_U @ r_ents
    return meas_all_2(l_ents, detect_model=many_mode_detector), meas_all_2(r_ents, detect_model=many_mode_detector)

In [ ]:
l_res, r_res = spoof_haar(10)
print(l_res)
print_all_coincidence_counts(l_res)
print_all_coincidence_counts(r_res)

AttributeError: 'tuple' object has no attribute 'T'

In [ ]:
print(ent(17,r=1))

NameError: name 'ent' is not defined